In [2]:
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms

import torch.utils.data
from torch.utils.data import DataLoader
from collections import Counter

from utils import EmotionsDataset, SimpleNet, AttentionalNet

In [3]:
# define dictionary for emotions
emotion_dict = {0: 'Angry', 
                1: 'Disgust', 
                2: 'Fear',
                3: 'Happy', 
                4: 'Sad',
                5: 'Surprise',
                6: 'Neutral'}

In [4]:
batch_size = 128
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5), (0.5))])
dataset_train = EmotionsDataset(root='./dataset', fname='icml_face_data_train.csv', transform=transform)
dataset_validation = EmotionsDataset(root='./dataset', fname='icml_face_data_validation.csv', transform=transform)
dataset_test = EmotionsDataset(root='./dataset', fname='icml_face_data_test.csv', transform=transform)

Loading dataset...	Done
Loading dataset...	Done
Loading dataset...	Done


In [5]:
torch.manual_seed(0)
trainloader = DataLoader(dataset_train, batch_size = batch_size,shuffle=True)
valloader = DataLoader(dataset_validation, batch_size = batch_size,shuffle=True)
testloader = DataLoader(dataset_test, batch_size = batch_size)

In [7]:
class Face_Emotion_CNN(nn.Module):
    def __init__(self):
        super(Face_Emotion_CNN, self).__init__()
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5,padding=3)
        self.cnn2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=4,padding=2)
        self.cnn3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5,padding=3)
        self.cnn4 = nn.Conv2d(in_channels=64, out_channels=128,kernel_size=5,padding=3)
        self.cnn5 = nn.Conv2d(in_channels=128, out_channels=128,kernel_size=4,padding=2)

        
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(3,stride=2)
        self.cnn1_bn = nn.BatchNorm2d(32)
        self.cnn2_bn = nn.BatchNorm2d(32)
        self.cnn3_bn = nn.BatchNorm2d(64)
        self.cnn4_bn = nn.BatchNorm2d(128)
        self.cnn5_bn = nn.BatchNorm2d(128)
       

        self.fc1 = nn.Linear(128*4*4, 1024)
        self.fc2 = nn.Linear(1024, 7)
        self.dropout = nn.Dropout(0.3)
        self.log_softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        x = self.relu(self.pool(self.cnn1_bn(self.cnn1(x))))
        #print(x.shape)
        x = self.relu(self.pool(self.cnn2_bn(self.cnn2(x))))
        #print(x.shape)
        x = self.relu(self.pool(self.cnn3_bn(self.cnn3(x))))
        #print(x.shape)
        x = self.relu(self.pool(self.cnn4_bn(self.cnn4(x))))
        #print(x.shape)
        x = self.relu((self.cnn5_bn(self.cnn5(x))))
        #print(x.shape)

        #x = x.view(x.size(0), -1)
        x = x.reshape(x.shape[0],128*4*4)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(self.dropout(x)))
        x = self.log_softmax(x)
        return x
    
torch.manual_seed(0)
model = Face_Emotion_CNN()

In [8]:
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

#class_count = [i for i in Counter(dataset_train.targets).values()]
#weight = torch.tensor(class_count) / sum(class_count)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

In [9]:
no_epochs = 20
training_loss = []
val_loss = []
val_acc = []
train_acc = []

for epoch in range(no_epochs): 
    scheduler.step()
    running_loss = 0.0
    running_loss_val = 0
    running_loss_train = 0
    correct_val = 0
    total_val = 0
    total_train = 0 
    correct_train = 0
    #print(f"Epoch {epoch+1}")
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        

        running_loss += loss.item()
        #if i % 2000 == 1999:    # print every 2000 mini-batches
            #print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')

    with torch.no_grad():
        for traindata in trainloader:
            trainimages, trainlabels = traindata
            trainoutputs = model(trainimages)
            loss_train = criterion(trainoutputs, trainlabels)
            running_loss_train += loss_train.item()
            train_, trainpredicted = torch.max(trainoutputs.data, 1)
            total_train += trainlabels.size(0)
            correct_train += (trainpredicted == trainlabels).sum().item()
            
    with torch.no_grad():
        for valdata in valloader:
            valimages, vallabels = valdata
            valoutputs = model(valimages)
            loss_val = criterion(valoutputs, vallabels)
            running_loss_val += loss_val.item()
            val_, valpredicted = torch.max(valoutputs.data, 1)
            total_val += vallabels.size(0)
            correct_val += (valpredicted == vallabels).sum().item()
    
    print(correct_val/total_val)
    training_loss.append(running_loss)
    val_loss.append(running_loss_val)
    val_acc.append(correct_val/total_val)
    train_acc.append(correct_train/total_train)

print('Finished Training')



C:\Users\User\anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
C:\Users\User\anaconda3\lib\site-packages\torch\autograd\__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(


0.4591808303148509
0.5121203677904709
0.5553078852047925
0.5608804680969629
0.5973808860406798
0.6040679855112845
0.6012816940651993
0.5943159654499861
0.6010030649205907
0.6082474226804123
0.6029534689328504
0.60490387294511
0.6035107272220674
0.5954304820284202
0.603789356366676
0.6060183895235441
0.6060183895235441
0.60490387294511
0.6068542769573697
0.60490387294511
Finished Training


In [12]:
train_acc

[0.46626493434114735,
 0.5470409975965725,
 0.598000626981086,
 0.6166010658678464,
 0.6830262287087673,
 0.6978996133616636,
 0.7245114772371033,
 0.7400815075411892,
 0.7628618203350865,
 0.7779790309658992,
 0.7816015883520847,
 0.7858511268243408,
 0.7880107283430283,
 0.7923995959455223,
 0.7945940297467693,
 0.7944198683339719,
 0.7966491344177784,
 0.7957086627886726,
 0.7969277926782542,
 0.7948726880072451]

In [64]:
PATH = './7_layer_val_nowd_norandomness.pth'
torch.save(model.state_dict(), PATH)

In [ ]:
#model = Face_Emotion_CNN()
model.load_state_dict(torch.load(PATH))

In [10]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct / total} %')

Accuracy of the network on the test images: 61.437726386179996 %


In [11]:
classes = ('Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral')

correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}


with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: Angry is 55.6 %
Accuracy for class: Disgust is 30.9 %
Accuracy for class: Fear  is 43.9 %
Accuracy for class: Happy is 84.3 %
Accuracy for class: Sad   is 48.5 %
Accuracy for class: Surprise is 70.9 %
Accuracy for class: Neutral is 57.3 %
